In [92]:
prompts = """
domes
windows
portals
spires
towers
minarets
portal
window
statue
tower
buttress
gargoyle
gate
relief
sculpture
Tympanum
Bell
transept
spire
atrium
dome
door
pediment
pillars
poles
colonnade
rose window
lancets
tympanum
jambfigures
minaret
arches
stair step crenellation
shady arcades
iwan
muqarnas
mihrab
jali
chapel
altar
organ
nave
choir
vault
chancel
ambulatory
Aisle
Ceiling
Painting
Triforium
apse
pulpit
chapels
arches
abside
stained glass windows
arches
roundels
minbar
chandelier
Spire
Atrium
Apse
Chevette
Pulpit
Dome
door
""".strip().splitlines()
# prompts = [f"a picture of a cathedral's {x}" for x in prompts]
prompts = [f"{x}" for x in prompts]
len(prompts)

67

In [93]:
prompts = list(dict.fromkeys(prompts))
prompts

['domes',
 'windows',
 'portals',
 'spires',
 'towers',
 'minarets',
 'portal',
 'window',
 'statue',
 'tower',
 'buttress',
 'gargoyle',
 'gate',
 'relief',
 'sculpture',
 'Tympanum',
 'Bell',
 'transept',
 'spire',
 'atrium',
 'dome',
 'door',
 'pediment',
 'pillars',
 'poles',
 'colonnade',
 'rose window',
 'lancets',
 'tympanum',
 'jambfigures',
 'minaret',
 'arches',
 'stair step crenellation',
 'shady arcades',
 'iwan',
 'muqarnas',
 'mihrab',
 'jali',
 'chapel',
 'altar',
 'organ',
 'nave',
 'choir',
 'vault',
 'chancel',
 'ambulatory',
 'Aisle',
 'Ceiling',
 'Painting',
 'Triforium',
 'apse',
 'pulpit',
 'chapels',
 'abside',
 'stained glass windows',
 'roundels',
 'minbar',
 'chandelier',
 'Spire',
 'Atrium',
 'Apse',
 'Chevette',
 'Pulpit',
 'Dome']

In [94]:
len(prompts)


64

In [95]:
DATA_DIR = '/home/cc/students/csguests/chendudai/Thesis/data/'
OUT_DIR = 'output'
CKPT = '/home/cc/students/csguests/chendudai/Thesis/data/clipseg-ft_model/0_CLIPModel'
CLIP_BASE = 'openai/clip-vit-base-patch32'

In [96]:
! mkdir {OUT_DIR}

mkdir: cannot create directory ‘output’: File exists


In [97]:
ls {DATA_DIR}

0148_megaDepth/
0_1_undistorted/
0209_megaDepth_mosque/
0209_mosque_geometric_occlusions.csv
0237_megaDepth_mosque/
62_0_undistorted/
98_0_undistorted/
98_3_undistorted/
badshahi_geometric_occlusions.csv
badshahi_mosque/
clipseg_ft_crops_10epochs/
clipseg-ft_model/
clip_sims_v0.3-notest24-nodups.csv
ft_clip_sims_v0.3-ft_bsz128_5epochs-lr1e-06-val091-2430-notest24-nodups.csv
hurba/
hurba_indoor/
manually_gt_masks_0_1/
manually_gt_masks_0209/
manually_gt_masks_badshahi/
manually_gt_masks_notre_dame/
manually_gt_masks_st_paul/
milano_base_clip.csv
milano_enamble_clip.csv
milano_geometric_clip.csv
milano_geometric_occlusions.csv
morris_npy/
notre_dame_base_clip.csv
notre_dame_clip_finetuned.csv
notre_dame_ensamble_clip.csv
notre_dame_front_facade/
notre_dame_geometric_clip.csv
notre_dame_geometric_occlusions.csv
per_label_metrics_old_without_a_picture_of_a_cathedrals.json
retrieval_clip_base_outdoor_070523.csv
retrieval_clip_base_outdoor_070523_old.csv
retrieval_clip_outdoor_020523-ft.csv


In [98]:
NAMES = ['0_1_undistorted', 'st_paul', 'notre_dame_front_facade','0209_megaDepth_mosque', 'badshahi_mosque', 'seville_indoor', 'sacre_coeur', 'hurba', 'hurba_indoor', '0237_megaDepth_mosque', '62_0_undistorted']

In [99]:
from glob import glob
from transformers import CLIPProcessor, CLIPModel
from PIL import Image
from tqdm.auto import tqdm
import torch
import numpy as np
import pandas as pd
import os

In [100]:
NAMES

['0_1_undistorted',
 'st_paul',
 'notre_dame_front_facade',
 '0209_megaDepth_mosque',
 'badshahi_mosque',
 'seville_indoor',
 'sacre_coeur',
 'hurba',
 'hurba_indoor',
 '0237_megaDepth_mosque',
 '62_0_undistorted']

In [101]:
%time proc = CLIPProcessor.from_pretrained(CLIP_BASE)

CPU times: user 99.8 ms, sys: 324 µs, total: 100 ms
Wall time: 3.28 s


In [102]:
# %time clip = CLIPModel.from_pretrained(CLIP_BASE)
%time clip = CLIPModel.from_pretrained(CKPT)


CPU times: user 652 ms, sys: 230 ms, total: 882 ms
Wall time: 676 ms


In [103]:
clip.to('cuda');
clip.eval();

In [104]:
fns = [
    fn
    for name in NAMES
    for fn in glob(f'{DATA_DIR}{name}/dense/images/*')
]
len(fns)

12699

In [105]:
fns[0]

'/home/cc/students/csguests/chendudai/Thesis/data/0_1_undistorted/dense/images/0620.jpg'

In [106]:
Ls = []
for fn in tqdm(fns):
    img = Image.open(fn).convert('RGB')
    with torch.no_grad():
        inp = proc(
            text=prompts,
            images=[img],
            return_tensors="pt",
            padding=True
        ).to('cuda')
        out = clip(**inp)
        L = out.logits_per_image / clip.logit_scale.exp()
        Ls.append(L.cpu().numpy())

  0%|          | 0/12699 [00:00<?, ?it/s]

In [107]:
LL = np.vstack(Ls).T
LL.shape

(64, 12699)

In [108]:
os.path.basename

<function posixpath.basename(p)>

In [109]:
df = pd.DataFrame({'fn': fns})
# df['fn'] = df.fn.str.extract('retrieval_occ/(.*)')[0]
df['base_fn'] = df.fn.apply(os.path.basename)
df['building'] = df.fn.str.extract('data/([^/]*)')[0]
for prompt, scores in zip(prompts, LL):
    df[prompt] = scores

In [110]:
df

,fn,base_fn,building,domes,windows,portals,spires,towers,minarets,portal,...,stained glass windows,roundels,minbar,chandelier,Spire,Atrium,Apse,Chevette,Pulpit,Dome
0,/home/cc/students/csguests/chendudai/Thesis/da...,0620.jpg,0_1_undistorted,-0.001470,0.128373,0.224818,0.148129,0.097729,-0.051262,0.160036,...,-0.034099,0.059388,-0.106820,0.030796,0.052044,0.044765,0.135725,0.035667,0.027424,0.015063
1,/home/cc/students/csguests/chendudai/Thesis/da...,0656.jpg,0_1_undistorted,-0.001105,0.127025,0.185777,0.139101,0.101438,-0.088349,0.141955,...,0.017885,0.076648,-0.068660,0.078066,0.050621,0.055633,0.214401,0.007192,0.058885,0.032301
2,/home/cc/students/csguests/chendudai/Thesis/da...,0049.jpg,0_1_undistorted,0.000953,0.128496,0.188316,0.192968,0.184378,-0.047847,0.124903,...,-0.015071,0.046207,-0.113907,0.059196,0.116344,0.083674,0.168105,0.034574,0.006138,0.000220
3,/home/cc/students/csguests/chendudai/Thesis/da...,0360.jpg,0_1_undistorted,-0.002212,0.120568,0.199293,0.138270,0.100842,-0.088170,0.147581,...,-0.003034,0.062533,-0.097032,0.062001,0.044839,0.069115,0.218438,0.033057,0.024417,0.018563
4,/home/cc/students/csguests/chendudai/Thesis/da...,0725.jpg,0_1_undistorted,0.011130,0.200158,0.357533,0.018204,0.079582,-0.065446,0.310843,...,-0.008696,0.103393,-0.046329,0.029407,0.012628,0.129936,0.201288,0.110233,0.136749,0.008309
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12694,/home/cc/students/csguests/chendudai/Thesis/da...,0548.JPG,62_0_undistorted,0.055474,0.114381,0.085013,0.053095,0.009701,-0.076686,0.034576,...,0.048067,0.187857,0.139322,0.233290,0.028205,0.139004,0.199262,0.097602,0.184037,0.203463
12695,/home/cc/students/csguests/chendudai/Thesis/da...,0577.jpg,62_0_undistorted,0.076232,0.107067,-0.048672,0.099706,-0.005842,-0.028976,-0.042931,...,-0.008162,0.111143,0.059856,0.284168,0.058702,0.157498,0.091916,0.088041,0.064632,0.251430
12696,/home/cc/students/csguests/chendudai/Thesis/da...,0260.JPG,62_0_undistorted,0.116968,0.122468,-0.058076,0.057019,-0.014917,-0.069035,-0.050116,...,0.022681,0.172063,0.020051,0.260332,0.044124,0.145379,0.142712,0.073722,0.034729,0.320640
12697,/home/cc/students/csguests/chendudai/Thesis/da...,0181.jpg,62_0_undistorted,0.022144,0.098872,0.040258,0.096567,0.017190,-0.075669,-0.016625,...,0.040365,0.114747,0.089320,0.128419,0.002468,0.078876,0.136906,0.129654,0.117321,0.125261


In [111]:
# df.to_csv(f'{OUT_DIR}/retrieval_clip_BASE_all_140523.csv', index=False)
df.to_csv(f'{OUT_DIR}/retrieval_clip_ft_all_140523.csv', index=False)
